## Setting

In [ ]:
from bs4 import BeautifulSoup
import datetime
from dotenv import load_dotenv
import os
import pathlib
import requests
import schedule
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
envpath = pathlib.Path('./.env')
load_dotenv(envpath)

In [153]:
# ヘッドレスモードで起動する
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# 誤ってdriverを複数起動しないようにする
try:
  driver.quit()
except:
  pass

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
amazon_url = "https://amzn.asia/d/134xdwa"
driver.get(amazon_url)
time.sleep(3)

## Main

### Nintendo-shop

In [ ]:
def get_soup(url):
  """
  urlからBeautifulSoupオブジェクトを取得する
  
  Parameters
  ----------
  url : str
    対象のURL
      
  Returns
  -------
  soup : BeautifulSoup
    BeautifulSoupオブジェクト
  """
  res = requests.get(url)
  time.sleep(3)
  return BeautifulSoup(res.text, 'html.parser')

In [ ]:
def nintendo_stock_checker(url):
  """
  urlから任天堂スイッチの在庫情報を取得する
  
  Parameters
  ----------
  url : str
    対象のURL
      
  Returns
  -------
  bool : bool
    在庫があるかどうか
  """
  soup = get_soup(url)
  item_name_elements = soup.select('.c-productList--item__name')
  for item_name_element in item_name_elements:
    if 'Nintendo Switch Proコントローラー' in item_name_element.text:
      return True
  return False

### amazon

In [ ]:
def get_price():
    """
    現在閲覧しているamazonの商品の価格を取得する
    
    Returns
    -------
    price : int
        商品の価格
    """
    driver.refresh()
    time.sleep(3)
    price_elements = driver.find_elements(By.CSS_SELECTOR, '#buybox .a-price-whole')
    if price_elements:
        price_element = price_elements[0]
        price = int(price_element.text.replace(',', ''))
        return price
    # 売ってなかったら高い値段を返す
    return 1e+10

### Line Notify

In [ ]:
def send_line_notify(notification_message):
    """
    LINEに通知する

    環境変数 YOUR_LINE_NOTIFY_TOKEN にトークンを設定しておくこと

    Parameters
    ----------
    notification_message : str
        通知するメッセージ

    Returns
    -------
    None
    """
    line_notify_token = os.environ.get('YOUR_NOTIFY_TOKEN')
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

In [ ]:
def main():
    global flag
    nintendo_url = 'https://store-jp.nintendo.com/list/hardware-accessory/controller/?prefn1=hasInventory&prefv1=true'
    if nintendo_stock_checker(nintendo_url):
        send_line_notify(f'在庫あり\nurl: {nintendo_url}')
        flag = True
        return True
    elif get_price() < 8000:
        send_line_notify('8000円以下になりました\n: url: {amazon_url}')
        flag = True
        return True

    print(f'{datetime.datetime.now}:処理を実行しました')

In [ ]:
schedule.clear()
schedule.every(3).minutes.do(main)

In [ ]:
flag = False
while True:
  if flag:
    break
  schedule.run_pending()
  time.sleep(50)